In [1]:
import pandas as pd
import csv
import re
import jieba
import time
from jieba import analyse
from collections import OrderedDict

In [2]:
all_lst = []
with open(r"D:\文档\天猫\store_of_Tmall_201808141052.csv", 'r', encoding='gbk') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        all_lst.append(row)

In [354]:
df = pd.read_excel(r"D:\文档\天猫\ME&CITY折扣服装.xls")

In [390]:
# df = pd.DataFrame(all_lst)
df.rename(columns={'标题': 'title', '名称':'store_name'}, inplace=True)
df.columns

Index(['store_name', 'ID ', 'title', '价格', '初始价格', '折后价格', '当月销量', '累计销量',
       '上市年份季节', '厚薄',
       ...
       'sell_img', 'none_brand_title', '小类', '中类', '大类', 'jieba', 'title简称',
       '名词', '性别', '季节'],
      dtype='object', length=121)

In [391]:
df['title'] = df['title'].str.replace('Polo衫', 'POLO衫')

In [392]:
df = df[(df['title'].str.find('婴儿') == -1) & (df['title'].str.find('童') == -1) & (~df['title'].str.isdigit())]
df = df[(df['title'].str.find('1元入会') == -1) & (df['title'].str.find('太平鸟A') == -1)]

In [393]:
jieba.load_userdict(r"D:\文档\天猫\服装字典")

In [394]:
df['title'] = df['title'].str.replace('【商场同款】', '')

In [395]:
# df['title'] = df['title'] .str.extract(brand_shop, expand=False)[2]
# df['title']
df['none_brand_title'] = df['title'].str.extract(brand_shop, expand=True)[1]
# df[df['none_brand_title'].isnull()][['none_brand_title','store_name', 'title']]

df.loc[df.index[df['none_brand_title'].isnull()], ['none_brand_title']] = df.loc[df.index[df['none_brand_title'].isnull()], ['title']].values
df.loc[df.index[df['none_brand_title'] == ''], ['none_brand_title']] = df.loc[df.index[df['none_brand_title'] == ''], ['title']].values
df.loc[df.index[df['none_brand_title'].str.count('') < 10], ['none_brand_title']] = df.loc[df.index[df['none_brand_title'].str.count('') < 10], ['title']].values

In [396]:
df['小类'] = ''
df['中类'] = ''
df['大类'] = ''

In [397]:
# df['title_rank'] = df['none_brand_title'].apply(lambda x: list(jieba.cut(x)))

In [398]:
# patt = r'裙|裤|袜|衫|马|连|腰|胸|环|护|毯|冰|西|羽绒|套|夹|链|鞋|牛仔|饰|围|袋|衣|恤|POLO|皮|克|帽|服|盒|包|带|结|褛|巾|棒|罩|雨|\
# 手机|心|袍|唐|靴|袖|镜|挂|枕|衬|阔腿|利领|床|杯|卫衣|风衣|毛衣|大衣|连衣|棉衣'
# df['title_rank'][1689]

In [399]:
# df[df['title简称'].str.find('牛仔') != -1]['title简称']
df['jieba'] = df['title'].apply(lambda x: list(jieba.posseg.cut(x)))

In [400]:
condition = df['jieba'].apply(lambda x: [i.word for i in x if i.flag == 'n' or i.flag == 'nr'])

In [401]:
df['title简称'] = (df['jieba'] .apply(lambda x: [i.word for i in x if i.flag == 'nz'])
                              .apply(lambda x: set(x))
                              .apply(lambda x: [i.replace('体恤', 'T恤') for i in set(x)])
                              .apply(lambda x: [i for i in x if x != ' ' and i.isalpha()])
                              .apply(lambda x: '|'.join(set(x))))

In [402]:
df['名词'] = (df['jieba'].apply(lambda x: [i.word for i in x if i.flag == 'n' or i.flag == 'nr'])
                       .apply(lambda x: set(x))
                       .apply(lambda x: [i.replace('体恤', 'T恤') for i in set(x)])
                       .apply(lambda x: [i for i in x if x != ' ' and i.isalpha()])
                       .apply(lambda x: '|'.join(set(x))))

In [403]:
rep_val = df.loc[df.index[(df['title简称'] == '')], ['名词']].values
df.loc[df.index[(df['title简称'] == '')], ['title简称']] = rep_val

In [404]:
# df.loc[(df['title简称'] != ''), ['title简称', 'title']]

In [405]:
# df['title简称'] = df['none_brand_title'].apply(lambda x: clothe_class.findall(x))

In [406]:
# def niuzai(lo):
#     if '牛仔' in lo:
#         if '短裤' in lo:
#             lo.append('牛仔短裤')
#         elif '长裤' in lo:
#             lo.append('牛仔长裤')
#         elif '中裤' in lo:
#             lo.append('牛仔中裤')
#         elif '休闲' in lo:
#             if '外套' in lo:
#                 lo.append('外套')
#             else:
#                 lo.append('牛仔长裤')        
#     return lo

In [407]:
# df['title简称'] = (df['title_rank'].apply(lambda x: set(x))
#                                 .apply(lambda x: [i.replace('体恤', 'T恤') for i in set(x)])
#                                 .apply(lambda x: [i for i in x if x != ' ' and i.isalpha()])
#                                 .apply(lambda x: niuzai(x))
#                                 .apply(lambda x: ''.join(set(x))))

In [408]:
# df['title简称'][1689]

#### 导入匹配规则表

In [425]:
mid_rank = []
mid_rank_dict = OrderedDict()
mid_rank_dict_2 = OrderedDict()
mid_rank_dict_3 = OrderedDict()
with open(r"D:\文档\天猫\rule_of_rank") as f:
    skirts = f.read().split('\n')
for one_rank in skirts:
    mid_rank.append(one_rank.split('\t'))
    mid_rank_dict[one_rank.split('\t')[0]] = one_rank.split('\t')[1]
    mid_rank_dict_2[one_rank.split('\t')[1]] = one_rank.split('\t')[2]
    mid_rank_dict_3[one_rank.split('\t')[2]] = one_rank.split('\t')[3]

In [426]:
for one_key in mid_rank:
    tmall = one_key[0]
    res = one_key[1]
    df.loc[(df['title简称'].str.find(tmall) != -1), ['小类']] = res

#### 生成中类和大类明细

In [427]:
# for one_key in mid_rank:
#     mid_res = one_key[1]
#     final_res = one_key[2]
#     df.loc[(df['中类'].str.find(mid_res) != -1), ['小类']] = final_res
df['中类'] = df['小类'].map(mid_rank_dict_2)

In [428]:
# for one_key in mid_rank:
#     mid_res = one_key[2]
#     final_res = one_key[3]
#     df.loc[(df['中类'].str.find(mid_res) != -1), ['大类']] = final_res
df['大类'] = df['中类'].map(mid_rank_dict_3)

In [429]:
sex_patt = re.compile(r'男女|男女同款|男|女')
quarter_patt = re.compile(r'春|夏|秋|冬|夏秋')

In [430]:
df['性别'] = (df['title'].apply(lambda x: sex_patt.findall(x))
#                                 .apply(lambda x: set(x))
                                .apply(lambda x: '|'.join(set(x)))
                                .replace('男|男女', '男女')
                                .replace('男|女', '男女'))
df['季节'] = (df['title'].apply(lambda x: quarter_patt.findall(x))
                                .apply(lambda x: set(x))
                                .apply(lambda x: '|'.join(set(x))))
# df['性别'].replace('男|男女', '男女', inplace=True)

#### 查看性别已经匹配成功的行

In [431]:
# df[((df['性别'] != '女') & (df['性别'] != '男') & (df['性别'] != ''))][['store_name', '性别', 'title']]
df['性别'].value_counts()
# df[(df['性别'] == '')][['store_name', '性别', 'title']]

女    213
男    162
       8
Name: 性别, dtype: int64

#### 对性别进行品牌匹配

In [432]:
with open(r"C:\Users\leshuangjiang\Desktop\ppp") as f:
    skirts = [i.split('\t') for i in f.read().split('\n')]
brand_sex = dict(skirts)

In [433]:
df['性别2'] = df['store_name'].map(brand_sex)
condition = ((df['性别'] != '女') & (df['性别'] != '男') & (df['性别'] != '男女'))
# condition

In [434]:
df.loc[df.index[condition], ['性别']] = df.loc[df.index[condition], ['性别2']].values

In [435]:
condition_2 = ((df['性别'] == '') & (df['中类'] == '鞋配'))
df.loc[df.index[condition_2], ['性别']] = '男女'
df.loc[df.index[((df['性别'] == '') & (df['中类'].str.find('裙') != -1))], ['性别']] = '男女'
# df.loc[df.index[df['性别'] == ''], ['store_name', 'title']]
df['性别'].value_counts()

女    213
男    162
Name: 性别, dtype: int64

In [436]:
df.columns

Index(['store_name', 'ID ', 'title', '价格', '初始价格', '折后价格', '当月销量', '累计销量',
       '上市年份季节', '厚薄',
       ...
       'none_brand_title', '小类', '中类', '大类', 'jieba', 'title简称', '名词', '性别',
       '季节', '性别2'],
      dtype='object', length=122)

#### 保存最终结果

In [437]:
# df[['分类', 'title简称']]
# df[df['分类'] == '']['title简称']
# new = df[['store_name', 'sold', 'title', '小类', '中类', '大类', 'title简称', '性别']]
new = df.drop(['性别2', 'jieba', '名词', 'none_brand_title'], axis=1)
new.to_excel('d:\\结果.xls')

In [383]:
# for one_item in df['none_brand_title']:
# #     string = '.'.join(one_item)
#     print(one_item)
#     print(clothe_class.findall(one_item))

In [699]:
# df.loc[(df['title简称'].apply(lambda x: '衫' in x)) & (df['分类'] == '')][['title', '分类']]
# df.loc[(df['title简称'].apply(lambda x: '克' in x)) & (df['分类'] == '')][['title','title简称']]

In [354]:
# clo_patt = re.compile(patt, re.I)

In [799]:
# df[df['分类'] != '']['分类']

In [122]:
brand_shop = r'(太平鸟|麦檬|MG小象|Material girl|me&city女装|me&city男装|美特斯邦威|npc|欧时力|ONLY|太平鸟女装|Aself|\
trendiano|优衣库|UR|VEROMODA|viishow|播|clot|速写|伊芙丽|lily|GU|GXG|gxgjeans|韩都衣舍|海澜之家|茵曼|杰克琼斯|JNBY|江南布衣|\
jucyjudy|kikc|kon|阔色|拉夏贝尔|lilbetter|Lilbetter|Libetter|LILBETTER|毛菇小象|Libette|MG|mg|MECITY|ASELF|TRENDIANO|\
Vero Moda|VIISHOW|gxg|HSPM|LB)(.*)'

In [128]:
pattern = r'O型裤|POLO|T恤|八分裤|包|保温杯|背带裤|背心|背心吊带|编织衫|便服|便裤|便西|踩脚裤|超短裤|衬衫|衬衣|床单|打底裤|打底衫|\
大衣|道袍|灯笼裤|低裆跨裤|吊带|短裤|短款裤装|短外套|短卫裤|短袖恤|耳环|耳饰|帆布|飞鼠裤|风衣|福袋|格子衫|工装裤|挂绳|哈伦裤|合体喇叭裤|\
护照夹|加绒裤|家居服|家居裤|夹克|紧身裤|颈链|九分裤|开衫|口罩|裤口褶边|裤子|垮裤|阔脚裤|阔腿|喇叭裤|蕾丝衫|礼服|连体|连体短裤|连体裤|\
裙|连衣裤|领带|领结|六分裤|萝卜裤|马夹|马甲|慢跑裤|毛巾|毛衫|毛衣|毛针织衫|帽|棉服|棉裤|棉褛|棉衣|抹胸|内裤|牛仔裤|皮带|皮衣|拼条裤|\
七分裤|铅笔裤|钱夹|茄克|晴雨伞|绒衫|沙滩裤|上衣|上装|手机|手链|束脚|束腿裤|双裤腰|睡裤|睡袍|睡衣|碎花衫|锁骨|唐装|套衫|套头衫|套装|体恤|\
条纹裤|头带|袜|外套|微喇叭裤|微喇加长裤|微喇裤|围巾|卫裤|卫衣|文胸|五分裤|西服|西服套装|西裤|西装|西装裤|线衫|项链|小脚|小脚长裤|小衫|鞋|\
胸针|休闲裤|修身裤装|靴|雪纺衫|腰带|腰封|荧光棒|羽绒服|雨衣|浴巾|圆领|运动|运动裤|运动衫|长裤|长款裤装|长卫裤|褶皱裤|针织裤|针织衫|正装裤|\
直筒|中裤|锥裤|锥形裤|拉链时尚高腰|棒球服|条纹短|底裤|内衣|镜|领巾|袋|挂件|抱枕|裤|宽腿裤|起居裤|蝴蝶衣|长衫|护腿|毯子|手套|护颈|围脖|\
连身裤|挂饰|扎脚裤|绑结腿裤|雪纺衬|女裤|披肩|摆件|恤|衬|夹|休闲|窄口裤|罩衫|牛仔|松紧裤|起居|手环|发带|拖|衫|花式裤|利领|西|冰袖|喇叭袖|\
瘦腿裤|长袖中长款|巾|半身群|围|T|荷叶边|露肩|套|休闲长裤|连衣裙|休闲短裤|休闲中裤|牛仔短裤|牛仔长裤|牛仔中裤'

In [129]:
# analyse.extract_tags()
# analyse.extract_tags()

In [130]:
clothe_class = re.compile(pattern, re.I)

In [375]:
# df.loc[df.index[df['title简称'].apply(lambda x: 'POLO' in x)], ['分类']] = 'POLO衫'
# df.loc[df.index[(df['title简称'].apply(lambda x: '恤' in x))  & (df['分类'] == '')], ['分类']] = 'T恤'
# df.loc[df.index[(df['title简称'].apply(lambda x: '衬' in x))  & (df['分类'] == '')], ['分类']] = '衬衫'
# df.loc[df.index[(df['title简称'].apply(lambda x: '卫衣' in x))  & (df['分类'] == '')], ['分类']] = '卫衣'
# df.loc[df.index[(df['title简称'].apply(lambda x: '毛衣' in x))  & (df['分类'] == '')], ['分类']] = '毛衣'
# df.loc[df.index[(df['title简称'].apply(lambda x: '克' in x and '夹' in x))  & (df['分类'] == '')], ['分类']] = '夹克'
# df.loc[df.index[(df['title简称'].apply(lambda x: '棉衣' in x))  & (df['分类'] == '')], ['分类']] = '棉衣'
# df.loc[df.index[(df['title简称'].apply(lambda x: '风衣' in x))  & (df['分类'] == '')], ['分类']] = '风衣'
# df.loc[df.index[(df['title简称'].apply(lambda x: '大衣' in x))  & (df['分类'] == '')], ['分类']] = '大衣'

In [801]:
pants_rank = {}
with open(r"D:\文档\天猫\pants") as f:
    pants = f.read().split('\n')
for one_rank in pants:
    pants_rank[one_rank] = '裤子'

In [771]:
pattern_2 = r'O型裤|POLO|T恤|八分裤|包|保温杯|背带裤|背心|背心吊带|编织衫|便服|便裤|便西|踩脚裤|超短裤|衬衫|衬衣|床单|打底裤|打底衫|\
大衣|道袍|灯笼裤|低裆跨裤|吊带|短裤|短款裤装|短外套|短卫裤|短袖恤|耳环|耳饰|帆布|飞鼠裤|风衣|福袋|格子衫|工装裤|挂绳|哈伦裤|合体喇叭裤|\
护照夹|加绒裤|家居服|家居裤|夹克|紧身裤|颈链|九分裤|开衫|口罩|裤口褶边|裤子|垮裤|阔脚裤|阔腿|喇叭裤|蕾丝衫|礼服|连体|连体短裤|连体裤|\
裙|连衣裤|领带|领结|六分裤|萝卜裤|马夹|马甲|慢跑裤|毛巾|毛衫|毛衣|毛针织衫|帽|棉服|棉裤|棉褛|棉衣|抹胸|内裤|牛仔裤|皮带|皮衣|拼条裤|\
七分裤|铅笔裤|钱夹|茄克|晴雨伞|绒衫|沙滩裤|上衣|上装|手机|手链|束脚|束腿裤|双裤腰|睡裤|睡袍|睡衣|碎花衫|锁骨|唐装|套衫|套头衫|套装|体恤|\
条纹裤|头带|袜|外套|微喇叭裤|微喇加长裤|微喇裤|围巾|卫裤|卫衣|文胸|五分裤|西服|西服套装|西裤|西装|西装裤|线衫|项链|小脚|小脚长裤|小衫|鞋|\
胸针|休闲裤|修身裤装|靴|雪纺衫|腰带|腰封|荧光棒|羽绒服|雨衣|浴巾|圆领|运动|运动裤|运动衫|长裤|长款裤装|长卫裤|褶皱裤|针织裤|针织衫|正装裤|\
直筒|中裤|锥裤|锥形裤|拉链时尚高腰|棒球服|条纹短|底裤|内衣|镜|领巾|袋|挂件|抱枕|裤|宽腿裤|起居裤|蝴蝶衣|长衫|护腿|毯子|手套|护颈|围脖|\
连身裤|挂饰|扎脚裤|绑结腿裤|雪纺衬|女裤|披肩|摆件|恤|衬|夹|窄口裤|罩衫|牛仔|松紧裤|起居|手环|发带|拖|衫|花式裤|利领|西|冰袖|喇叭袖|\
瘦腿裤|长袖中长款|巾|半身群|围|T|荷叶边|露肩|套'

In [772]:
dfff = pd.DataFrame(pattern_2.split('|'))

In [773]:
dfff.to_excel('d:\\fenlei.xls')

In [459]:
lo

['休闲裤', '束腿裤', '运动', '裤', '九分裤', '短裤']

['男装',
 '2018',
 '夏',
 '九分',
 '休闲',
 '休闲裤',
 '束',
 '腿',
 '裤',
 '薄款',
 '刺绣',
 '微弹',
 '青年',
 '青年运动',
 '运动',
 '运动裤',
 '九分',
 '九分裤']

In [1146]:
# 

In [ ]:
# s.isin(['婴儿'])

In [ ]:
# s[s]

In [396]:
p = r'A字裙|O型裤|POLO|T恤|八分裤|百褶裙|半裙|半身裙|包|包臀裙|保温杯|背带裤|背带裙|背心|背心吊带|编织衫|便服|便裤|便西|\
不规则裙|踩脚裤|超短裤|超短裙|衬衫|衬衣|床单|打底裤|打底衫|大衣|蛋糕裙|道袍|灯笼裤|灯笼裙|低裆跨裤|吊带|短裤|短款裤装|短裙|短外套|\
短卫裤|短袖恤|耳环|耳饰|帆布|飞鼠裤|风衣|福袋|高腰裙|格子衫|工装裤|公主裙|挂绳|哈伦裤|合体喇叭裤|荷叶边裙|护照夹|花苞裙|及膝裙|加绒裤|\
家居服|家居裤|夹克|紧身裤|颈链|九分裤|开衫|口罩|裤口褶边|裤子|垮裤|阔脚裤|阔腿|喇叭裤|蕾丝衫|礼服|连身裙|连体|连体短裤|连体裤|连衣|\
连衣裤|连衣裙|领带|领结|六分裤|萝卜裤|马夹|马甲|慢跑裤|毛巾|毛衫|毛衣|毛针织衫|帽|棉服|棉裤|棉褛|棉衣|抹胸|内裤|牛仔裤|牛仔裙|蓬蓬裙|\
皮带|皮衣|拼条裤|七分裤|七分裙|铅笔裤|铅笔裙|钱夹|茄克|晴雨伞|裙裤|绒衫|伞裙|沙滩裤|纱裙|上衣|上装|手机|手链|束脚|束腿裤|双裤腰|睡裤|\
睡袍|睡衣|碎花衫|锁骨|唐装|套裙|套衫|套头衫|套装|体恤|条纹裤|头带|袜|外套|微喇叭裤|微喇加长裤|微喇裤|围巾|卫裤|卫衣|文胸|五分裤|五分裙|\
西服|西服套装|西裤|西装|西装裤|西装裙|仙女裙|线衫|项链|小黑裙|小脚|小脚长裤|小衫|鞋|胸针|休闲裤|修身裤装|靴|雪纺衫|腰带|腰封|\
一步裙|荧光棒|羽绒服|雨衣|浴巾|圆领|运动黑色韩版裤|运动裤|运动衫|长裤|长款裤装|长裙|长筒裙|长卫裤|褶皱裤|针织裤|针织衫|正装裤|直筒|直筒裤|\
直筒裤装|直筒长裤|中裤|中裙|中长裙|锥裤|锥形裤|拉链时尚高腰|棒球服|鱼尾裙|透视裙|条纹短|底裤|内衣|镜|领巾|袋|挂件|碎花裙|抱枕|太平鸟A|\
休闲男裤|绣花裙|大摆裙|宽腿裤|起居裤|迷你裙|蕾丝裙|窄身裙|喇叭裙|打褶裙|排扣裙|紧身裙|罗纹裙|弹力裙|蝴蝶衣|裙子|长衫|护腿|毯子|手套|护颈|\
围脖|连身裤|挂饰|扎脚裤|绑结腿裤|雪纺衬|女裤|披肩|摆件|T|衬|夹|窄口裤|睡裙|罩衫|牛仔|松紧裤|起居|手环|发带|拖|衫|花式裤|利领|西|冰袖|\
喇叭袖|瘦腿裤|腰裙|长袖中长款|丝巾|半身群|针织裙|围领|雪纺裙'

In [398]:
pf = pd.DataFrame(p.split('|'))

In [399]:
pf.to_excel(r'd:\\fenlei2.xls')

In [146]:
k = 'sd4'
k.isalpha()

False